In [8]:
import os
import re
from scipy import ndimage, misc
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import img_to_array
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

from skimage.transform import resize, rescale
import matplotlib.pyplot as plt
import numpy as np
np. random. seed(0)
import cv2 as cv2

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense ,Conv2D,MaxPooling2D ,Dropout
from tensorflow.keras.layers import Conv2DTranspose, UpSampling2D, add
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
from tensorflow.keras.utils import plot_model
import tensorflow as tf

In [ ]:
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)',key)]
    return sorted(data,key = alphanum_key)
# defining the size of the image
SIZE = 256
high_img = []
path = '/content/drive/MyDrive/Super_Resolution/Data/Raw Data/high_res'
files = os.listdir(path)
files = sorted_alphanumeric(files)
for i in tqdm(files):
    if i == '855.jpg':
        break
    else:
        img = cv2.imread(path + '/'+i,1)
        # open cv reads images in BGR format so we have to convert it to RGB
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        #resizing image
        img = cv2.resize(img, (SIZE, SIZE))
        img = img.astype('float32') / 255.0
        high_img.append(img_to_array(img))


low_img = []
path = '/content/drive/MyDrive/Super_Resolution/Data/Raw Data/low_res'
files = os.listdir(path)
files = sorted_alphanumeric(files)
for i in tqdm(files):
    if i == '855.jpg':
        break
    else:
        img = cv2.imread(path + '/'+i,1)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        #resizing image
        img = cv2.resize(img, (SIZE, SIZE))
        img = img.astype('float32') / 255.0
        low_img.append(img_to_array(img))

In [ ]:
# for checking whether data is loaded correctly, not necessary to run this cell
'''
for i in range(4):
    a = np.random.randint(0,855)
    plt.figure(figsize=(10,10))
    plt.subplot(1,2,1)
    plt.title('High Resolution Imge', color = 'green', fontsize = 20)
    plt.imshow(high_img[a])
    plt.axis('off')
    plt.subplot(1,2,2)
    plt.title('low Resolution Image ', color = 'black', fontsize = 20)
    plt.imshow(low_img[a])
    plt.axis('off')
'''

In [ ]:
train_high_image = high_img[:700]
train_low_image = low_img[:700]
train_high_image = np.reshape(train_high_image,(len(train_high_image),SIZE,SIZE,3))
train_low_image = np.reshape(train_low_image,(len(train_low_image),SIZE,SIZE,3))

validation_high_image = high_img[700:830]
validation_low_image = low_img[700:830]
validation_high_image= np.reshape(validation_high_image,(len(validation_high_image),SIZE,SIZE,3))
validation_low_image = np.reshape(validation_low_image,(len(validation_low_image),SIZE,SIZE,3))


test_high_image = high_img[830:]
test_low_image = low_img[830:]
test_high_image= np.reshape(test_high_image,(len(test_high_image),SIZE,SIZE,3))
test_low_image = np.reshape(test_low_image,(len(test_low_image),SIZE,SIZE,3))

print("Shape of training images:",train_high_image.shape)
print("Shape of test images:",test_high_image.shape)
print("Shape of validation images:",validation_high_image.shape)

In [ ]:
from keras import layers
from tensorflow.keras.utils import plot_model

input_img=Input(shape=(256,256,3))

l1 = Conv2D(60, 3, padding='same', activation='relu')(input_img)

l2 = Conv2D(30, 5, padding='same', activation='relu')(l1)

l3 = Conv2D(16, 1, padding='same', activation='relu')(l2)

l4 = Conv2D(10, 3, padding='same', activation='relu')(l3)

l5 = Conv2D(3, 5, padding='same', activation='relu')(l4)

SRCNN = Model(input_img, l5)

def pixel_mse_loss(x,y):
    return tf.reduce_mean( (x - y) ** 2 )
SRCNN.compile(optimizer=tf.keras.optimizers.Adam(0.001),loss=pixel_mse_loss)
SRCNN.summary()
plot_model(SRCNN, to_file ='super_res.png',show_shapes=True)

In [ ]:
SRCNN.fit(train_low_image, train_high_image, epochs=10, batch_size=1,
              validation_data=(validation_low_image, validation_high_image))

In [16]:
def PSNR(y_true, y_pred):
    mse = tf.reduce_mean((y_true - y_pred) ** 2)
    psnr = 20 * tf.math.log(1 / tf.sqrt(mse)) / tf.math.log(10.0)
    return psnr
def pixel_mse_loss(x,y):
    return tf.reduce_mean( (x - y) ** 2 )
SRCNN.compile(optimizer=tf.keras.optimizers.Adam(0.01),loss=pixel_mse_loss)

In [ ]:

def plot_images(high,low,predicted):
    plt.figure(figsize=(15,15))
    plt.subplot(1,3,1)
    plt.title('High Image', color = 'green', fontsize = 20)
    plt.imshow(high)
    plt.subplot(1,3,2)
    plt.title('Low Image ', color = 'black', fontsize = 20)
    plt.imshow(low)
    plt.subplot(1,3,3)
    plt.title('Predicted Image ', color = 'Red', fontsize = 20)
    plt.imshow(predicted)

    plt.show()
SIZE=256
epoch=0
ssim_history = []
psnr_history = []
for i in range(1,25):

    predicted = np.clip(SRCNN.predict(test_low_image[i].reshape(1,SIZE, SIZE,3)),0.0,1.0).reshape(SIZE, SIZE,3)
    img = predicted
    blur=cv2.GaussianBlur(img, (1, 1), 0)
    mask=cv2.subtract(img,blur)
    predicted =cv2.add(img,mask)
    predicted = cv2.GaussianBlur(predicted, (3, 3), 0)

    plot_images(test_high_image[i],test_low_image[i],predicted)
    epoch_psnr = PSNR(test_high_image[i], predicted)
    epoch_ssim = tf.image.ssim(test_high_image[i],predicted,max_val=1)
    # Append value to history
    psnr_history.append(epoch_psnr)
    ssim_history.append(epoch_ssim)

    print("Epoch {}: Validation PSNR = {}".format(epoch + 1, epoch_psnr))
    print('PSNR',PSNR(test_high_image[i],predicted),'dB', "SSIM",tf.image.ssim(test_high_image[i],predicted,max_val=1))
    epoch+=1


In [ ]:

# Create a figure with subplots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))

# Plot PSNR values
ax1.plot(range(1, epoch + 1), psnr_history, marker='o', label='PSNR')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('PSNR')
ax1.set_title('PSNR vs. Epoch')
ax1.grid(True)
ax1.legend()

# Plot SSIM values
ax2.plot(range(1, epoch + 1), ssim_history, marker='o', label='SSIM')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('SSIM')
ax2.set_title('SSIM vs. Epoch')
ax2.grid(True)
ax2.legend()

# Adjust layout to prevent overlap
plt.tight_layout()

# Show the combined plot
plt.show()


In [ ]:
from google.colab import files
from PIL import Image
import cv2
# Upload image from local desktop
uploaded = files.upload()

# Get the uploaded image file name
image_filename = list(uploaded.keys())[0]

# Load the image using PIL
img = Image.open(image_filename)

In [ ]:
# Display the image
plt.imshow(img)
plt.show()

In [23]:
new_filename = 'chk.png'  # Change the name as needed
img.save(new_filename)

In [24]:
img = cv2.imread('chk.png')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#img = downsample_image(img,4)
#resizing image
img = cv2.resize(img, (SIZE, SIZE))
img = img.astype('float32') / 255.0
#img = downsample_image(img,2)
low_img.append(img_to_array(img))

In [ ]:
predicted = np.clip(SRCNN.predict(img.reshape(1,SIZE, SIZE,3)),0.0,1.0).reshape(SIZE, SIZE,3)
img = predicted
blur=cv2.GaussianBlur(img, (1, 1), 0)
mask=cv2.subtract(img,blur)
predicted =cv2.add(img,mask)
predicted = cv2.GaussianBlur(predicted, (3, 3), 0)

plt.figure(figsize=(15,15))
plt.subplot(1,2,1)
plt.title('Low Image ', color = 'black', fontsize = 20)
plt.imshow(img)
plt.subplot(1,2,2)
plt.title('Predicted Image ', color = 'Red', fontsize = 20)
plt.imshow(predicted)

plt.show()